In [18]:
#import Libraries

import numpy as np
import pandas as pd
import re

In [19]:
#First import the dataset in the dataset variable
#data_review variable contains the reveiws and ratings 
dataset = pd.read_csv("zomato.csv")
data_review = dataset['reviews_list']

print(dataset['reviews_list'][0])

MemoryError: 

In [4]:
x = []
y = []

In [5]:
# here we tokenize the rating string and the review string
#loop over all the rows
for row_num in range(0,51717):
    # split the revie text at '()
    lst = data_review[row_num].split("('")
    for i in lst:
        if len(i) > 5:
            if i.find("',") != -1:
                single_rev = i.split("',")
                if len(single_rev[0]) > 2:
                    x.append(single_rev[0])
                if len(single_rev[1]) > 2:    
                    y.append(single_rev[1])



In [ ]:
#!pip install nltk

In [6]:
#Import the Librariesimport re 
import nltk
nltk.download("stopwords") 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alekhya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# to store the final rating
rating_final = []
# to store cleaned revies
review_final = []


In [8]:
# the rating string contains words and numbers 
# so we tokenize the numbers only from it and change into float
# for rartings below 2.5 we store the rating as poor
# for ratings between 2.5 and 3.5 the rating as average
# for ratings more than 3.5 tha rating stored as good

for loop in range(0,40000):
    data_x = x[loop]
    data_x = re.sub('[a-zA-Z]', " ", data_x)
    data_x = data_x.split()
    data_x = ''.join(data_x)
    data_x = float(data_x)
    if data_x < 2.5:
        rating_final.append("poor") #poor
    elif data_x >= 2.5 and data_x <= 3.5 :    
        rating_final.append("average") # average
    elif data_x > 3.5:
        rating_final.append("good") #good
        

In [9]:
!pip install sklearn

In [10]:
# label encode the Ratings and OneHotEncode for the classification        

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
rating_final = le.fit_transform(rating_final)

rating_final = np.array(rating_final)
rating_final = np.expand_dims(rating_final, axis=1)
        
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
rates = one.fit_transform(rating_final).toarray()

In [11]:
# Here the unnecessary stop words from the reviews lists
# Stemming operations are also done here 

for loop in range(0,40000) : 
    data_y = y[loop]
    data_y = re.sub('[^a-zA-Z]', " ", data_y)
    data_y = data_y.lower()
    data_y = data_y.split()
    data_y = [ps.stem(word) for word in data_y if not word in set(stopwords.words('english'))]
    data_y = ' '.join(data_y)
    review_final.append(data_y)

In [12]:
# count vectorize the reviews according to the unique words    
                    
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 20000)
x_final = cv.fit_transform(review_final).toarray()

In [13]:
# saving the vectorizer which would be used as dictionary.
import pickle
pickle.dump(cv, open('cv.pkl','wb'))

In [14]:
# Split the data into test and train sets

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final,rates, test_size = 0.2, random_state = 0)  

In [ ]:
# adding the neuron layers 
# the units in the input layers is equal to the number of unique words
# taken three deeper layers of 2000 units each
# Relu as activation in the hidden layers
# the output layer has 3 units as the one hot encoding has 3 columns
# the classification is in categorical


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units = 13264, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 3, kernel_initializer = 'random_uniform', activation = 'softmax'))
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size = 128,epochs = 20)

In [ ]:
# testing the prediction

y_pred = model.predict(x_test)

text =  "The food is okay. average place "
text = re.sub('[^a-zA-Z]', ' ',text)
text = text.lower()
text = text.split()
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text = ' '.join(text)

y_p = model.predict(cv.transform([text]))

In [ ]:
model.save("zomato_2_analysis.h5")